# Неделя 5. Computer Vision
## Generative Adversarial Networks

### ‼️ Загрузи этот ноутбук на Google Colab

Реализуй генеративно состязательную сеть, которая будет генерировать изображения на произвольном датасете. 

Можно использовать: 
* [Abstract Art Gallery](https://www.kaggle.com/datasets/bryanb/abstract-art-gallery) - абстрактные картины
* [Fake Faces](https://www.kaggle.com/datasets/hyperclaw79/fakefaces) - лица, сгенерированные нейросетью StyleGAN2
* [Anime Faces](https://www.kaggle.com/datasets/soumikrakshit/anime-faces) - лица в стиле героев anime
* [Animal Faces](https://www.kaggle.com/datasets/andrewmvd/animal-faces) - морды животных (можно взять какой-нибудь один класс)
* [Cat Faces](https://www.kaggle.com/datasets/spandan2/cats-faces-64x64-for-generative-models) - морды котов (из демонстрационного примера)

# 🥸

### Рекомендации

* Начни с очень простых архитектур, первая цель - убедиться, что модель технически: совпадают размерности, процесс обучения не вылетает с ошибкой и т.д. 
* Быстрого результата ожидать не стоит: обычно нужно подождать 10-20 эпох
* Сохраняй полученные изображения, потом их можно будет объединить в gif и анимировать!


# 📍 Задание 1

Модифицировать GAN в Wasserstein GAN по следующей формуле: ![](https://miro.medium.com/max/1400/1*5jF5gbIDwU6k9m1ILl0Utg.jpeg)

🔖 Подсказка `keras`: [How to Develop a Wasserstein Generative Adversarial Network (WGAN) From Scratch](https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/)

🔖 Подсказка для `pyTorch`: [PyTorch-GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan/wgan.py)

Обрати внимание на: 
1. Метки для настоящих и сгенерированных изображений
2. Лосс-функции
3. Клиппинг весов
4. Архитектуру дискриминатора (для **WGAN** - *«критика»*)
5. Число итераций по критику и по генератору
6. Используемый оптимизатор и learning rate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
from torch import device
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/kaggle.json /root/.kaggle

In [ ]:
!ls ~/.kaggle/

In [ ]:
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! !mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

/bin/bash: !mkdir: command not found


In [ ]:
!kaggle datasets download -d spandan2/cats-faces-64x64-for-generative-models

 96% 92.0M/96.0M [00:00<00:00, 94.3MB/s]
100% 96.0M/96.0M [00:00<00:00, 103MB/s] 


In [ ]:
! kaggle datasets download -d andrewmvd/animal-faces

 99% 691M/696M [00:12<00:00, 75.9MB/s]
100% 696M/696M [00:12<00:00, 59.0MB/s]


In [ ]:
!unzip -qq cats-faces-64x64-for-generative-models.zip -d cats/

In [ ]:
! rm -rf afhq

In [ ]:
!unzip -qq animal-faces.zip

In [ ]:
! rm -rf afhq/train/dog
! rm -rf afhq/train/wild

In [ ]:
image_size = 64
batch_size = 128
latent_size= 64
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

train = ImageFolder('/content/drive/MyDrive/Colab Notebooks/bouquetes/train', transform=tt.Compose([tt.Resize(image_size),
                                                  tt.ToTensor(),
                                                  tt.Normalize(*stats)]))
 
train_dl = DataLoader(train, batch_size, shuffle=True) 

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.upsample2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)            
        )

        self.upsample3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)            
        )

        self.upsample4 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)    
        )

        self.upsample5 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        x = self.upsample5(x)

        return x


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)            
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True) 
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Flatten(),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x



In [ ]:
generator = Generator()
summary(generator.to(device), (64, 1, 1), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [128, 512, 4, 4]         524,800
       BatchNorm2d-2           [128, 512, 4, 4]           1,024
              ReLU-3           [128, 512, 4, 4]               0
   ConvTranspose2d-4           [128, 256, 8, 8]       2,097,152
       BatchNorm2d-5           [128, 256, 8, 8]             512
              ReLU-6           [128, 256, 8, 8]               0
   ConvTranspose2d-7         [128, 128, 16, 16]         524,288
       BatchNorm2d-8         [128, 128, 16, 16]             256
              ReLU-9         [128, 128, 16, 16]               0
  ConvTranspose2d-10          [128, 64, 32, 32]         131,072
      BatchNorm2d-11          [128, 64, 32, 32]             128
             ReLU-12          [128, 64, 32, 32]               0
  ConvTranspose2d-13           [128, 3, 64, 64]           3,072
             Tanh-14           [128, 3,

In [ ]:
discriminator = Discriminator()
summary(discriminator.to(device), (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
       BatchNorm2d-2           [-1, 64, 32, 32]             128
         LeakyReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 16, 16]         131,072
       BatchNorm2d-5          [-1, 128, 16, 16]             256
         LeakyReLU-6          [-1, 128, 16, 16]               0
            Conv2d-7            [-1, 256, 8, 8]         524,288
       BatchNorm2d-8            [-1, 256, 8, 8]             512
         LeakyReLU-9            [-1, 256, 8, 8]               0
           Conv2d-10            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-11            [-1, 512, 4, 4]           1,024
        LeakyReLU-12            [-1, 512, 4, 4]               0
           Conv2d-13              [-1, 1, 1, 1]           8,192
          Flatten-14                   

In [ ]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

In [ ]:
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors).to(device)
    fake_fname = 'image-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [ ]:
def train_discriminator(real_images, opt_d):
    # обнуляем градиенты у дискриминатора, сейчас мы будем его учить
    opt_d.zero_grad()

    # получаем предсказания дискриминатора на настоящих картинках
    real_preds = discriminator(real_images).to(device) 

    # делаем вектор меток для настоящих картинок - он состоит из единиц
    # real_targets = torch.ones(real_images.size(0), 1).to(device) 
    real_targets = torch.ones(real_images.size(0), 1).to(device) 

    # считаем лосс между предсказаниями и настоящими метками
    # real_loss = torch.log(real_preds)
    real_loss = F.binary_cross_entropy(real_preds, real_targets) 

    # записываем усредненные предсказания дискриминатора
    real_score = torch.mean(real_preds).item()
    
    # создаем случайный шум: из него генератор нам что-нибудь нагенерирует
    latent = torch.randn(batch_size, latent_size, 1, 1).to(device) 

    # получаем фейковые изображения
    fake_images = generator(latent).to(device)  

    # делаем вектор меток класса для фейковых изображений
    fake_targets = torch.zeros(fake_images.size(0), 1).to(device) 

    # получаем предсказания дискриминатора для фейковых изображений
    fake_preds = discriminator(fake_images).to(device)  

    # считаем лосс на фейковых изображениях
    # fake_loss = torch.log(1 - fake_preds)
    fake_loss = F.binary_cross_entropy(fake_preds, fake_targets)  

    # записываем усредненные предсказания дискриминатора на фейках
    fake_score = torch.mean(fake_preds).item()

    # итоговый лосс - сумма двух лоссов
    loss = torch.mean(real_loss + fake_loss)

    # делаем обратный проход
    loss.backward()

    # обновляем веса дискриминатора
    opt_d.step()

    return loss.item(), real_score, fake_score

In [ ]:
def train_generator(opt_g):
    # обнуляем градиенты оптимизатора генератора 
    opt_g.zero_grad()

    # генерируем случайный вектор для генератора
    latent = torch.randn(batch_size, latent_size, 1,1).to(device) 

    # получаем фейковые изображения
    fake_images = generator(latent).to(device) 
    
    # получаем предсказания дискриминатора для фейковых изображений
    preds = discriminator(fake_images).to(device) 

    # делаем вектор меток класса - теперь они единицы для фейковых картинок
    targets = torch.ones(batch_size, 1).to(device)

    # считаем лосс
    # loss = torch.mean(torch.log(1 - preds))
    loss = F.binary_cross_entropy(preds, targets) 
    
    # делаем обратный проход
    loss.backward()

    # обновляем веса генератора
    opt_g.step()
    
    return loss.item(), latent

In [ ]:
def fit(epochs=10, lr=0.0004, start_idx=131):
    torch.cuda.empty_cache()
    
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
  
    opt_d = torch.optim.Adam(discriminator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            
            real_images = real_images.to(device)

            loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            loss_g, latent = train_generator(opt_g)
            
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        print(f"[{epoch+1}/{epochs}], loss_g: {loss_g:.4f}, loss_d: {loss_d:.4f}, real_score: {real_score:.4f}, fake_score: {fake_score:.4f}")
    
        save_samples(epoch+start_idx, latent, show=False)
        if epoch % 5 == 0:
            torch.save(discriminator, '/content/drive/MyDrive/Colab Notebooks/discriminator_bouq.pt')
            torch.save(generator, '/content/drive/MyDrive/Colab Notebooks/generator_bouq.pt')
    
    return losses_g, losses_d, latent, fake_scores

In [ ]:
len(train_dl)

2

In [ ]:
loss_g, loss_dc, latent, fake_scores = fit(epochs=200)

  0%|          | 0/2 [00:00<?, ?it/s]

[1/200], loss_g: 6.0002, loss_d: 3.5656, real_score: 0.9843, fake_score: 0.9166
Saving image-0031.png


  0%|          | 0/2 [00:00<?, ?it/s]

[2/200], loss_g: 3.6975, loss_d: 0.3822, real_score: 0.7705, fake_score: 0.0783
Saving image-0032.png


  0%|          | 0/2 [00:00<?, ?it/s]

...

[198/200], loss_g: 5.5302, loss_d: 0.1278, real_score: 0.9135, fake_score: 0.0310
Saving image-0228.png


  0%|          | 0/2 [00:00<?, ?it/s]

[199/200], loss_g: 5.3504, loss_d: 0.1929, real_score: 0.8648, fake_score: 0.0349
Saving image-0229.png


  0%|          | 0/2 [00:00<?, ?it/s]

[200/200], loss_g: 5.3549, loss_d: 0.0644, real_score: 0.9575, fake_score: 0.0187
Saving image-0230.png


In [ ]:
! cp -r /content/generated /content/drive/MyDrive/Colab\ Notebooks/gen_bouq

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__.


In [ ]:
# code

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.upsample2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)            
        )

        self.upsample3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)            
        )

        self.upsample4 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)    
        )

        self.upsample5 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        x = self.upsample5(x)

        return x


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)            
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True) 
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 32, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Flatten(),
            nn.Linear(32, 1)
            # nn.Tanh()
            # nn.Sigmoid()
        )
            

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x



In [ ]:
generator = Generator()
summary(generator.to(device), (64, 1, 1), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [128, 512, 4, 4]         524,800
       BatchNorm2d-2           [128, 512, 4, 4]           1,024
              ReLU-3           [128, 512, 4, 4]               0
   ConvTranspose2d-4           [128, 256, 8, 8]       2,097,152
       BatchNorm2d-5           [128, 256, 8, 8]             512
              ReLU-6           [128, 256, 8, 8]               0
   ConvTranspose2d-7         [128, 128, 16, 16]         524,288
       BatchNorm2d-8         [128, 128, 16, 16]             256
              ReLU-9         [128, 128, 16, 16]               0
  ConvTranspose2d-10          [128, 64, 32, 32]         131,072
      BatchNorm2d-11          [128, 64, 32, 32]             128
             ReLU-12          [128, 64, 32, 32]               0
  ConvTranspose2d-13           [128, 3, 64, 64]           3,072
             Tanh-14           [128, 3,

In [ ]:
discriminator = Discriminator()
summary(discriminator.to(device), (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
       BatchNorm2d-2           [-1, 64, 32, 32]             128
         LeakyReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 16, 16]         131,072
       BatchNorm2d-5          [-1, 128, 16, 16]             256
         LeakyReLU-6          [-1, 128, 16, 16]               0
            Conv2d-7            [-1, 256, 8, 8]         524,288
       BatchNorm2d-8            [-1, 256, 8, 8]             512
         LeakyReLU-9            [-1, 256, 8, 8]               0
           Conv2d-10            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-11            [-1, 512, 4, 4]           1,024
        LeakyReLU-12            [-1, 512, 4, 4]               0
           Conv2d-13             [-1, 32, 1, 1]         262,144
          Flatten-14                   

In [ ]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

In [ ]:
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)

def save_samples(index, latent_tensors, show=True):
    fake_images = generator(latent_tensors).to(device)
    fake_fname = 'image-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [ ]:
clip_value = 0.01

In [ ]:
def train_discriminator(real_images, opt_d):
    # обнуляем градиенты у дискриминатора, сейчас мы будем его учить
    opt_d.zero_grad()

    # получаем предсказания дискриминатора на настоящих картинках
    real_preds = discriminator(real_images).to(device) 

    # делаем вектор меток для настоящих картинок - он состоит из единиц
    # real_targets = torch.ones(real_images.size(0), 1).to(device) 
    # real_targets = torch.ones(real_images.size(0), 1).to(device) 

    # считаем лосс между предсказаниями и настоящими метками
    # real_loss = torch.log(real_preds)
    real_loss = -torch.mean(real_preds)
    #  + torch.mean(real_targets) 
    

    # записываем усредненные предсказания дискриминатора
    real_score = torch.mean(real_preds).item()
    
    # создаем случайный шум: из него генератор нам что-нибудь нагенерирует
    latent = torch.randn(batch_size, latent_size, 1, 1).to(device) 

    # получаем фейковые изображения
    fake_images = generator(latent).to(device)  

    # делаем вектор меток класса для фейковых изображений
    # fake_targets = torch.zeros(fake_images.size(0), 1).to(device) 

    # получаем предсказания дискриминатора для фейковых изображений
    fake_preds = discriminator(fake_images).to(device)  

    # считаем лосс на фейковых изображениях
    # fake_loss = torch.log(1 - fake_preds)
    fake_loss = torch.mean(fake_preds)
    #  + torch.mean(fake_preds)
      

    # записываем усредненные предсказания дискриминатора на фейках
    fake_score = torch.mean(fake_preds).item()

    # итоговый лосс - сумма двух лоссов
    loss = torch.mean(real_loss + fake_loss)

    # делаем обратный проход
    loss.backward()

    # обновляем веса дискриминатора
    opt_d.step()

    for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)

    return loss.item(), real_score, fake_score

In [ ]:
def train_generator(opt_g):
    # обнуляем градиенты оптимизатора генератора 
    opt_g.zero_grad()

    # генерируем случайный вектор для генератора
    latent = torch.randn(batch_size, latent_size, 1,1).to(device) 

    # получаем фейковые изображения
    fake_images = generator(latent).to(device) 
    
    # получаем предсказания дискриминатора для фейковых изображений
    preds = discriminator(fake_images).to(device) 

    # делаем вектор меток класса - теперь они единицы для фейковых картинок
    targets = torch.ones(batch_size, 1).to(device)

    # считаем лосс
    # loss = torch.mean(torch.log(1 - preds))
    loss = -torch.mean(preds) 
    
    # делаем обратный проход
    loss.backward()

    # обновляем веса генератора
    opt_g.step()
    
    return loss.item(), latent

In [ ]:
def fit(epochs=5, lr=0.0005, crit = 2, start_idx=6):
    torch.cuda.empty_cache()
    
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
  
    opt_d = torch.optim.RMSprop(generator.to(device).parameters(), lr=lr)
    # opt_d = torch.optim.Adam(discriminator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.RMSprop(discriminator.to(device).parameters(), lr=lr)
    # opt_g = torch.optim.Adam(generator.to(device).parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            
            real_images = real_images.to(device)
            for i in range(crit):
                loss_d, real_score, fake_score = train_discriminator(real_images, opt_d)
            loss_g, latent = train_generator(opt_g)
            
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        print(f"[{epoch+1}/{epochs}], loss_g: {loss_g:.4f}, loss_d: {loss_d:.4f}, real_score: {real_score:.4f}, fake_score: {fake_score:.4f}")
    
        save_samples(epoch+start_idx, latent, show=False)
        if epoch % 5 == 0:
            torch.save(discriminator, '/content/drive/MyDrive/Colab Notebooks/was_discriminator_cat.pt')
            torch.save(generator, '/content/drive/MyDrive/Colab Notebooks/was_generator_cat.pt')
    
    return losses_g, losses_d, latent, fake_scores

In [ ]:
loss_g, loss_dc, latent, fake_scores = fit(epochs=5)

  0%|          | 0/41 [00:00<?, ?it/s]

[1/5], loss_g: -0.2817, loss_d: 0.0006, real_score: 0.2811, fake_score: 0.2817
Saving image-0006.png


  0%|          | 0/41 [00:00<?, ?it/s]

[2/5], loss_g: -0.2854, loss_d: 0.0007, real_score: 0.2847, fake_score: 0.2854
Saving image-0007.png


  0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
! pip install gradio

In [ ]:
import torch
from torch import nn
import os
from torchvision.utils import save_image
import gradio as gr


latent_size=64


# задаем класс модели 

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True)
        )

        self.upsample2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True)            
        )

        self.upsample3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True)            
        )

        self.upsample4 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)    
        )

        self.upsample5 = nn.Sequential(
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        x = self.upsample5(x)

        return x

# функция генерации: на вход число картинок
# на выходе имя файла
def generate(number):
    global gen
    gen.eval()
    noise = torch.randn((number, 64, 1, 1))
    tensors = gen(noise)
    save_image(tensors, 'cats.jpg', normalize=True)
    return 'cats.jpg'


# инициализация модели: архитектура + веса    
def init_model():
    global gen 
    gen = Generator()
    gen.load_state_dict(weight)
    return gen

# запуск gradio
def run(share=True):    
    gr.Interface(
        generate,
        inputs=[gr.inputs.Slider(label='Number of cats', minimum=8, maximum=32, step=8, default=8)],
        outputs="image",
    ).launch(share=share)

    
if __name__ == '__main__':
    init_model()
    run()

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://13069.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


In [ ]:
weight = torch.load('/content/drive/MyDrive/Colab Notebooks/generator_cat.pt', map_location=torch.device('cpu')).state_dict()

## 📍Задание 2

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/83/Telegram_2019_Logo.svg/640px-Telegram_2019_Logo.svg.png" height=32>
Реализуй telegram-бота, которому пользователь может прислать целое число, а обученный генератор сети генерирует заданное число картинок. 
Не забудь ограничить максимальное число картинок (например, 20), и сделать проверку на отрицательные числа. 

In [ ]:
# code

## 📍 Задание 3
Реализуй Conditional (W)GAN на датасете [Animal Faces](https://www.kaggle.com/datasets/andrewmvd/animal-faces). Можно использовать два класса: котов и собак. Тогда условием генерации для генератора будет вектор класса. 

![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F793761%2Fafb6d1aa3be4c82e17a0d0b802436e82%2Fafhq_dataset.jpg?generation=1590130743209322&alt=media)

Схема условной генеративно-состязательной сети: 

![](https://www.spiedigitallibrary.org/ContentImages/Proceedings/11584/115841U/FigureImages/00046_PSISDG11584_115841U_page_4_4.jpg)

In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(4, 4)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


# Loss functions
adversarial_loss = torch.nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

In [ ]:
# code

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__. 


In [ ]:
# code